In [1]:
%load_ext autoreload
%autoreload 2
import data_loader
from feature_extraction import extract_features, data_in_tw
import matplotlib.pyplot as plt

#from datetime import datetime, timedelta
import pandas as pd
import pytz
import pickle

from sklearn.svm import SVC

from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import resample

from sklearn.metrics import mean_squared_error
import numpy as np




In [2]:
# Size of time window in unit of second.
TIME_WINDOW_SEC = 10
features = []
labels = []


# Data Loading

In [3]:
dl = data_loader.DataLoader()

# Load Groudtruth
df_labels = dl.load_groundtruth()


# Load low frequency data
df_location = dl.load_low_freq_data('LOCATION')
df_activity = dl.load_low_freq_data('ACTIVITY_TYPE')



Loading /Users/zxxia/Research/bharath-data/lowfreqdata/10_null_LOCATION_null_PHONE_null_null_org.md2k.phonesensor_null_DATA.csv...
Loading /Users/zxxia/Research/bharath-data/lowfreqdata/11_null_ACTIVITY_TYPE_null_PHONE_null_null_org.md2k.phonesensor_null_DATA.csv...


In [4]:
visited = set()
_, _, beacon_id_set = dl.load_datasources()

for index, row in df_labels.iterrows():
    d = row['Start Timestamp'].date()
    if d not in visited:
        # load high frequency data
        df_battery = dl.load_high_freq_data('BATTERY', d)
        df_compass = dl.load_high_freq_data('COMPASS', d)
        df_ambient_light = dl.load_high_freq_data('AMBIENT_LIGHT', d)
        beacon_dfs = []
        for beacon_id in beacon_id_set:
            beacon_dfs.append(dl.load_high_freq_data('BEACON', d, beacon_id))
        if not beacon_dfs:
            df_beacon = None
        else:
            df_beacon = pd.concat(beacon_dfs)
            df_beacon.sort_values(by=['Timestamp'])
        
        visited.add(d)

    start = row['Start Timestamp']
    end = row['End Timestamp']

    if start >= end:
        continue

    # Obtain data needed for one label(one event)
    streams = dict()
    streams['Battery'] = data_in_tw(df_battery, start, end)
    streams['Compass'] = data_in_tw(df_compass, start, end)
    streams['Activity'] = data_in_tw(df_activity, start, end)
    streams['Location'] = data_in_tw(df_location, start, end)
    streams['Ambient Light'] = data_in_tw(df_ambient_light, start, end)
    streams['Beacon'] = data_in_tw(df_beacon, start, end)

    # Segment data streams into time windows, extract features and append features
    tmp = extract_features(streams, start, end, TIME_WINDOW_SEC)
    features = features + tmp
    
    # Extract labels and append labels
    labels = labels + [row['Labels']] * len(tmp)

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012000_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012001_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012002_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012003_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012004_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012005_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012006_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012007_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012008_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012009_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012010_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012011_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012012_9.csv.gz...
Loading /Users/zxxia/Rese

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012121_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012122_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012123_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012111_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012112_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012113_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012114_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012115_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012116_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012117_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012118_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012119_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012120_14.csv.gz...
Loadi

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012200_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012201_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012202_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012203_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012204_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012205_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012206_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012207_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012208_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012209_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012210_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012211_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012212_9.csv.gz...
Loading /Users/zxxia/Rese

Loading /Users/zxxia/Research/bharath-data/raw/raw57/2018012220_57.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw56/2018012220_56.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw442/2018012220_442.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw443/2018012220_443.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw445/2018012220_445.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw444/2018012220_444.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw439/2018012220_439.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw441/2018012220_441.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw448/2018012220_448.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012200_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012201_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/2018012202_219.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw219/20

Loading /Users/zxxia/Research/bharath-data/raw/raw220/2018012310_220.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw220/2018012311_220.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw220/2018012312_220.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw220/2018012323_220.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012300_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012301_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012302_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012303_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012304_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012305_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012306_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012307_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw22

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012400_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012401_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012402_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012403_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012404_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012405_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012406_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012407_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012408_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012409_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012410_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012411_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012412_9.csv.gz...
Loading /Users/zxxia/Rese

Loading /Users/zxxia/Research/bharath-data/raw/raw222/2018012423_222.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw501/2018012415_501.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw519/2018012423_519.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012400_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012401_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012402_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012403_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012404_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012405_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012406_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012407_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw221/2018012419_221.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw22

Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012503_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012504_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012505_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012506_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012507_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012508_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012509_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012510_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012511_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012512_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012513_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw223/2018012514_223.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw22

In [5]:
print(len(features))
print(len(labels))



45535
45535


# Data Preprocessing

In [6]:
# Convert features and labels to pandas dataframe format
df_features = pd.DataFrame(features)
df_features['Labels'] = pd.DataFrame(labels)
df_features.to_csv('bharath_features.csv')



df_features['Battery Charging'].interpolate(method='nearest', inplace=True)

df_features['Max GPS Accuracy'].fillna(1000, inplace=True)
#df_features['Max GPS Accuracy'].interpolate(method='nearest', inplace=True)
#df_features['Max GPS Speed'].interpolate(method='nearest', inplace=True)
#df_features.drop('Mean Battery Temperature', axis=1, inplace=True)

df_features.drop('Magnetic Field Variance', axis=1, inplace=True)
df_features = df_features[df_features['Beacon']!=1]
df_features = df_features[df_features['IN_VEHICLE']!=1]
df_features = df_features[df_features['ON_BICYCLE']!=1]
df_features.drop('Beacon', axis=1, inplace=True)
df_features.drop('IN_VEHICLE', axis=1, inplace=True)
df_features.drop('ON_BICYCLE', axis=1, inplace=True)
df_features.drop('UNKNOWN', axis=1, inplace=True)
df_features.drop('Max GPS Speed', axis=1, inplace=True)
df_features.dropna(axis=0, inplace=True)

# Separate entire dataframe into different classes
df_incar = df_features[df_features.Labels=='Incar']
df_indoor = df_features[df_features.Labels=='Indoor']
df_outdoor = df_features[df_features.Labels=='Outdoor']




In [217]:
pd.set_option('display.max_rows', 1000)
display(df_features)


,Battery Charging,Beacon,IN_VEHICLE,Max GPS Accuracy,Max GPS Speed,Max Light Intensity,ON_BICYCLE,ON_FOOT,RUNNING,STILL,TILTING,Time of day,UNKNOWN,WALKING,Labels
0,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
1,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
2,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
3,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
4,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
5,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
6,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
7,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor
8,NaN,0,0,NaN,NaN,0.151993,0,0,0,1,0,0,0,0,Indoor
9,NaN,0,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,Indoor


In [7]:
print(df_indoor.shape)
print(df_outdoor.shape)
print(df_incar.shape)
print(df_labels.shape)
print(df_features.shape)


(2695, 10)
(583, 10)
(187, 10)
(107, 5)
(3465, 10)


In [8]:
# Downsample majority class

df_indoor = resample(df_indoor, 
                     replace=583,   # sample without replacement
                     n_samples=583,   # to match minority class
                     random_state=42) # reproducible results

df_features = pd.concat([df_indoor, df_outdoor]) #df_incar

df_labels = df_features['Labels']
df_features.drop('Labels', axis=1, inplace=True)


# train_features is numpy array
vec = DictVectorizer()
train_features = vec.fit_transform(df_features.to_dict('records')).toarray()

# Normalize columns of features with min-max scaler
scaler = MinMaxScaler(copy=False)
train_features = scaler.fit_transform(train_features)

# Encode all features
le = LabelEncoder()
# le.fit(['Incar', 'Indoor', 'Outdoor'])
le.fit(['Indoor', 'Outdoor'])
train_labels = le.transform(df_labels)



In [ ]:
def predict_labels(feature_frame):
    labels = []
    for index, row in feature_frame.iterrows():
        if row['IN_VEHICLE']:
            labels.append('Incar')
        elif row['ON_BICYCLE']:
            labels.append('Outdoor')
        elif row['Beacon']:
            labels.append('Indoor') 
        else:
            # TODO: use classifier to predict
            pass
    return labels
    
    

# Training on SVM

## Tuning SVM Parameters

In [ ]:
clf = SVC()
param_grid = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 
     'kernel' : ['rbf','poly', 'sigmoid'], 
     'probability' : [True],
     'gamma' : [0.1]}
]
grid = GridSearchCV(clf, param_grid, cv=10, scoring='accuracy')
grid.fit(train_features, train_labels)
print("Grid scores on development set:")

means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print("Best parameters:", grid.best_params_)



In [273]:
# 10 fold corss validation
clf = SVC(C=10, kernel='sigmoid')

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    confusionmatrix = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:\n', confusionmatrix)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Micro F1 Score:', f1_score(y_test, y_pred, average='micro'))
    print('Macro F1 Score:', f1_score(y_test, y_pred, average='macro'))
    print('Weighted F1 Score:', f1_score(y_test, y_pred, average='weighted'))
    print('\n')


Confusion Matrix:
 [[54  9]
 [18 36]]
Accuracy: 0.769230769231
Micro F1 Score: 0.769230769231
Macro F1 Score: 0.763636363636
Weighted F1 Score: 0.766433566434


Confusion Matrix:
 [[52  7]
 [23 35]]
Accuracy: 0.74358974359
Micro F1 Score: 0.74358974359
Macro F1 Score: 0.738059701493
Weighted F1 Score: 0.738384998086


Confusion Matrix:
 [[59  7]
 [13 38]]
Accuracy: 0.82905982906
Micro F1 Score: 0.82905982906
Macro F1 Score: 0.823369565217
Weighted F1 Score: 0.827434039391


Confusion Matrix:
 [[50  6]
 [27 34]]
Accuracy: 0.717948717949
Micro F1 Score: 0.717948717949
Macro F1 Score: 0.71257351299
Weighted F1 Score: 0.710893761441


Confusion Matrix:
 [[59  4]
 [21 33]]
Accuracy: 0.786324786325
Micro F1 Score: 0.786324786325
Macro F1 Score: 0.775224775225
Weighted F1 Score: 0.779067086759


Confusion Matrix:
 [[55  4]
 [21 37]]
Accuracy: 0.786324786325
Micro F1 Score: 0.786324786325
Macro F1 Score: 0.781144781145
Weighted F1 Score: 0.78143255921


Confusion Matrix:
 [[47 11]
 [20 38]]
Ac

In [9]:
# save the model to disk
filename = 'bhararh_model.clf'
pickle.dump(clf, open(filename, 'wb'))



In [16]:
from hmmlearn.hmm import GaussianHMM
model = GaussianHMM(n_components=3)
model.fit(train_features)
pred_labels = model.predict(train_features)
for y_pred, y_true in zip(pred_labels, train_labels):
    print(y_pred, y_true)

/Users/zxxia/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/zxxia/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/zxxia/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/zxxia/anaconda3/lib/python3.6/site-package

2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
0 1
2 1
2 1
2 1
2 1
0 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
0 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
0 1
2 1
2 1
2 1
2 1
2 1
2 1
1 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
1 1
2 1
2 1
2 1
2 1
2 1
2 1
0 1
1 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
0 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
0 1
2 1
1 1
2 1
0 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
0 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
1 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
1 1
2 1
2 1
2 1
0 1
2 1
2 1
2 1
2 1
1 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1


In [25]:
svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
svd.fit_transform(train_features)
print(svd.explained_variance_ratio_)
print(train_features.shape)


ValueError: n_components must be < n_features; got 17 >= 17